# **Global Wheat Deetction**

# Intro

## Darknet with yolo v4
CSPDarknet53 contains 29 convolutional layers 3×3, a 725×725 receptive field and 27.6 M parameters. </br>
Darknet is mainly for Object Detection, and have different architecture, features than other deep learning frameworks. It is faster than many other NN architectures and approaches like FasterRCNN etc. You have to be in C if you need speed, and most of the deep nn frameworks are written in c. I would say Tensorflow has a broader scope, but Darknet architecture & YOLO is a specialized framework, and they are on top of their game in speed and accuracy. YOLO can run on CPU but you get 500 times more speed on GPU as it leverages CUDA and cuDNN.

## What Is CUDA?
CUDA is a parallel computing platform and programming model that makes using a GPU for general purpose computing simple.
used with C.


# Installation

### Imports

In [1]:
# importing libraries
import os
import csv
import json
from glob import glob
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image
import matplotlib.patches as patches
from bokeh.models import ColumnDataSource, HoverTool, Panel
from bokeh.plotting import figure
from bokeh.io import output_notebook, show, output_file
import cv2
import albumentations as albu
from tensorflow.keras.models import load_model
from albumentations.pytorch.transforms import ToTensor
from albumentations.core.transforms_interface import DualTransform
from albumentations.augmentations.bbox_utils import denormalize_bbox, normalize_bbox

### copy darknet & data

In [2]:
# clone darknet repo from AlexeyAB and data
!mkdir /kaggle/temp
!cp -r /kaggle/input/darknet-files /kaggle/temp/darknet
!cp -r /kaggle/input/libcuda/libcuda /kaggle/temp

### Download weights & configurations from drive

In [3]:
!mkdir -p /kaggle/temp/global-wheat-detection-data/weights
!cp /kaggle/input/detection-data/yolov4-obj_best.weights /kaggle/temp/global-wheat-detection-data/weights

In [4]:
!cp /kaggle/input/detection-data/obj.data /kaggle/temp/global-wheat-detection-data
!cp /kaggle/input/detection-data/obj.names /kaggle/temp/global-wheat-detection-data
!cp /kaggle/input/detection-data/yolov4-obj.cfg /kaggle/temp/global-wheat-detection-data

### fix libcuda & edit MakeFile

In [5]:
#fix libcuda problem
!rm /usr/lib/x86_64-linux-gnu/libcuda.so
!cp -as /kaggle/temp/libcuda/libcuda.so /usr/lib/x86_64-linux-gnu/libcuda.so
!rm /usr/lib/x86_64-linux-gnu/libcuda.so.1
!cp -as /kaggle/temp/libcuda/libcuda.so.1 /usr/lib/x86_64-linux-gnu/libcuda.so.1
# change makefile to have GPU and OPENCV
%cd /kaggle/temp/darknet
!sed -i 's/OPENCV=0/OPENCV=1/' Makefile
!sed -i 's/GPU=0/GPU=1/' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/' Makefile
!sed -i 's/CUDNN_HALF=0/CUDNN_HALF=1/' Makefile

### Make

In [6]:
# make darknet
# darknet executable file help us run and train yolo model
%cd /kaggle/temp/darknet
!make

# Define helper functions

In [7]:
# define helper functions from repo owner
def imShow(path):
  import cv2
  import matplotlib.pyplot as plt
  %matplotlib inline

  image = cv2.imread(path)
  height, width = image.shape[:2]
  resized_image = cv2.resize(image,(3*width, 3*height), interpolation = cv2.INTER_CUBIC)

  fig = plt.gcf()
  fig.set_size_inches(18, 10)
  plt.axis("off")
  plt.imshow(cv2.cvtColor(resized_image, cv2.COLOR_BGR2RGB))
  plt.show()

In [8]:
def create_txt_for_images(input_path, output_path):
    image_files = []
    for filename in os.listdir(input_path):
        if filename.endswith(".jpg"):
            image_files.append(os.path.join(input_path, filename))
    with open(output_path, "w") as outfile:
        for image in image_files:
            outfile.write(image)
            outfile.write("\n")

In [9]:
def format_output(input_json, output_csv):
    with open(input_json) as json_file:
        images = json.load(json_file)

    with open(output_csv, 'w', newline='') as csv_file:
        csv_writer = csv.writer(csv_file)
        csv_writer.writerow(['image_id', 'PredictionString'])

        for image in images:
            image_id = os.path.basename(image['filename']).split('.')[0]
            prediction = []
            for b_box in image['objects']:
                confidence = b_box['confidence']
                coordinates = b_box['relative_coordinates']
                for key, val in coordinates.items():
                    coordinates[key] *= 1024
                coordinates['center_x'] = coordinates['center_x'] - coordinates['width'] / 2
                coordinates['center_y'] = coordinates['center_y'] - coordinates['height'] / 2
                for key, val in coordinates.items():
                    coordinates[key] = round(coordinates[key])
                prediction.append(str(confidence))
                prediction.extend(map(str, coordinates.values()))
            csv_writer.writerow([image_id, ' '.join(prediction)])

## Move all files you need to complete testing process

In [10]:
# upload the custom .cfg from drive to darknet files
!mv /kaggle/temp/global-wheat-detection-data/yolov4-obj.cfg /kaggle/temp/darknet/cfg
# upload the obj.names and obj.data from drive to darknet files
!mv /kaggle/temp/global-wheat-detection-data/obj.names /kaggle/temp/darknet/data
!mv /kaggle/temp/global-wheat-detection-data/obj.data  /kaggle/temp/darknet/data

## test your model

### edit darknet configurations

In [11]:
# need to set our custom cfg to test mode 
%cd /kaggle/temp/darknet/cfg
!sed -i 's/batch=64/batch=1/' yolov4-obj.cfg
!sed -i 's/subdivisions=16/subdivisions=1/' yolov4-obj.cfg

### Test one image

In [12]:
# run your custom detector with this command (upload an image to your google drive to test, thresh flag sets accuracy that detection must be in order to show it)
%cd /kaggle/temp/darknet
!./darknet detector test data/obj.data cfg/yolov4-obj.cfg /kaggle/temp/global-wheat-detection-data/weights/yolov4-obj_best.weights /kaggle/input/global-wheat-detection/test/aac893a91.jpg -thresh 0.5
imShow('predictions.jpg')

### Run test set

In [13]:
%cd /kaggle/temp/darknet
create_txt_for_images("/kaggle/input/global-wheat-detection/test", "/kaggle/temp/global-wheat-detection-data/test.txt")
!./darknet detector test data/obj.data cfg/yolov4-obj.cfg /kaggle/temp/global-wheat-detection-data/weights/yolov4-obj_best.weights -ext_output -dont_show -thresh 0.5 -out /kaggle/temp/global-wheat-detection-data/result.json < /kaggle/temp/global-wheat-detection-data/test.txt

In [14]:
format_output('/kaggle/temp/global-wheat-detection-data/result.json','/kaggle/working/submission.csv')